In [2]:
import requests
import json
import pandas as pd
import math
import os
from IPython.display import Image, display, clear_output


In [3]:
def readJson(file):
    with open(file,"r") as f:
        rf=json.load(f)
    return rf

def readTsv(file):
    with open(file, "r") as f:
        rf=csv.DictReader(f, dialect='excel-tab')
    return rf

In [7]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]
elev_key = config["elevation_key"]
test_area = config["test_area"]

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

In [8]:
# data = readJson("mapillary_data.json")
data = pd.read_csv("training_data/features_no_dupes.csv").drop_duplicates(subset="id",keep="first").set_index("id")
data

,avg_speed,min_speed,max_speed,max_accel,max_deaccel,med_speed,med_accel,sd_speed,sd_accel,percent_ped_speed,percent_bic_speed,percent_car_speed,num_stops,sinuosity,distance,duration
id,,,,,,,,,,,,,,,,
6lmxhlsdu2820pp9ewk41z,16.092217995524045,0.232964,25.961550,10.652202,-13.501058,12.864293,12.076630,5.964283,2.795336,0.697674,6.279070,86.511628,0.0,1.037741,7917.451715,492.005
g234mtreigui4a64i4xea9,6.322339061546877,0.141193,12.872192,3.198201,-6.585177,6.365499,4.891689,2.824493,1.406421,0.735294,20.588235,36.764706,2.0,1.007190,1132.324604,179.099
hwt93drmgmyk3pnt7hib08,1.2753090252536683,0.620147,2.196088,1.129826,-7.976933,0.787971,4.553379,0.274747,0.971887,67.193676,0.000000,0.000000,0.0,2.232828,366.273853,287.204
9gmnn1cdfv7vbptl9p83u2,1.2108235182424214,0.079203,2.395365,1.275051,-4.657094,1.158081,2.966073,0.336322,0.656864,60.103627,0.000000,0.000000,4.0,1.304693,562.095759,464.226
18ejk5cjxixj051qiupxyb,1.3132966068373433,0.637032,2.017232,1.256552,-2.369025,0.690100,1.812788,0.291555,0.574907,57.831325,0.000000,0.000000,0.0,1.040797,126.595226,96.395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
frlAsAiloH7sIbIHy4ZyAg,1.4848487465561462,1.100335,2.089263,1.990085,-0.470809,0.494464,1.230447,0.299528,0.525397,38.095238,0.000000,0.000000,0.0,1.000800,233.121253,157.000
KbVGvM_dpBZidNPlXwOMnQ,1.3210914820317117,0.869768,1.805226,1.448281,-0.345011,0.467729,0.896646,0.240208,0.426121,64.705882,0.000000,0.000000,0.0,1.064708,158.530978,120.000
k9FvSmiv2eRKtfCP48gGzA,1.2378869177309548,0.305195,2.456294,2.003500,-0.292721,1.075550,1.148110,0.448620,0.513731,39.534884,0.000000,0.000000,0.0,1.003015,269.859348,218.000


In [14]:
f = open("sequence_labels_no_dupes.csv", "a+")
fs = open("skipped_segments_no_dupes.csv","a+")
index_to_start = pd.read_csv("sequence_labels_no_dupes.csv", header=None).shape[0] if os.stat("sequence_labels_no_dupes.csv").st_size != 0 else 0
num_skipped = pd.read_csv("skipped_segments_no_dupes.csv", header=None).shape[0] if os.stat("skipped_segments_no_dupes.csv").st_size != 0 else 0
index_to_start = index_to_start+num_skipped
a = ""
num = data.shape[0]
num_imgs = 10


for i, seq in enumerate(data.index[index_to_start:], index_to_start):
    images = list(reversed(requestImagesBySequence(seq)["features"]))
    interval = math.ceil(len(images)/num_imgs) if len(images)<50 else 10
    if(a=="q"):
        break
    j = 0
    incr = 1
    while(True):
        clear_output()
        print(f"[INFO]\nprogress:{i}/{num}\n")
        print(j+interval, len(images))
        key = images[j]["properties"]["key"]
        display(Image(url= f"https://images.mapillary.com/{key}/thumb-320.jpg"))
        a = input(f"0 = ped, 1 = bic, 2 = car, q=quit, anything_else = next image\n{data.iloc[i]}\n")
        if(a=="q"):
            break
        elif(len(a)>0 and a=="s"):
            fs.write(f"{seq}\n")
            break
        elif(len(a)>0 and a=="i"):
            if(interval==math.ceil(len(images)/num_imgs) or interval==10):
                interval=1
            else:
                interval = math.ceil(len(images)/num_imgs) if len(images)<50 else 10
        elif(len(a)>0 and a=="f"):
            images = list(reversed(images))
            j = 0
        elif(len(a)>0 and (int(a)==0 or int(a)==1 or int(a)==2)):
            f.write(f"{a},{seq}\n")
            break
        else: 
            if(j+interval>len(images)-1):
                if(incr==len(images)-1):
                    j = 0
                    incr = 1
                else:
                    j = incr
                    incr+=1
                print("More images")
            else:
                j+=interval

f.close()
fs.close()

[INFO]
progress:0/1870

60 431


KeyboardInterrupt: 